# 通过PYNQ加速OPENCV函数(Sobel算子)

在阅读本部分UserGuide时,请确认已做好以下准备：
*   已经按照之前的预备文档安装好依赖环境
*   2根HDMI传输线
*   一台支持HDMI的显示器

## 步骤1：加载cv2pynq库

In [ ]:
import cv2pynq as cv2

在正常运行的情况下,可以看到PYNQ板卡标记为“DONE”的LED闪烁(为加载了bit文件的效果);   
这是由于在封装的时候,我们在初始化阶段调用了Overlay方法给PYNQ加载了定制的bit文件:

>def __init__(self, load_overlay=True):     
&ensp;&ensp;&ensp;&ensp;self.bitstream_name = None  
&ensp;&ensp;&ensp;&ensp;self.bitstream_name = "cv2pynq03.bit"  
&ensp;&ensp;&ensp;&ensp;self.bitstream_path =os.path.join(CV2PYNQ_BIT_DIR,self.bitstream_name)<br>
&ensp;&ensp;&ensp;&ensp;self.ol = Overlay(self.bitstream_path)

上述代码为cv2pynq.py的部分节选，从当中可以看出在初始化的过程中,加载了cv2pynq03.bit，  
因此在导入库的时候会出现加载bit文件的效果。加载的bit文件的Block Design如下图所示:

![Image1](../image/1.png)

这个Block Design主要由以下三个部分组成：
* HDMI输入输出模块（移植于BaseOverlay）
* 由Vivado HLS生成的OPENCV算法加速IP核（内嵌于Image_Fliter模块）
* 基于AXI总线架构的流传输通道

## 步骤2：实例化HDMI输入输出接口

在进行cv2pynq的测试之前,我们需要引入图片/视频流，<br>此处由Block Design中设置好的HDMI输入模块传入视频以及输出模块输出经过处理好的视频流信息;<br>关于HDMI输入输出的更多详情，可以参考BaseOverlay中的Video模块;<br>https://github.com/Xilinx/PYNQ/tree/master/boards/Pynq-Z2/base/notebooks/video
<br>https://github.com/Xilinx/PYNQ/tree/master/boards/Pynq-Z1/base/notebooks/video

In [ ]:
hdmi_in = cv2.video.hdmi_in
hdmi_out = cv2.video.hdmi_out

hdmi_in.configure(cv2.PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

print(hdmi_in.mode)

在正确的输入视频流信息之后，我们可以得到输入视频流的配置信息；<br>在本实验中,最大支持 1920 * 1080的输入信号。

## 步骤3：采用原始的OPENCV中的Sobel算子对输入信号进行处理

导入原始的OPENCV

In [ ]:
import cv2 as openCV

导入time模块计算输出的帧率

In [ ]:
import time

设置开始时间、结束时间以及迭代次数，通过原始的OPENCV对输入信号进行处理

In [ ]:
import cv2 as openCV
import time

iterations = 10

start = time.time()
for i in range(iterations):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    openCV.Sobel(inframe,-1,1,0,ksize=5,dst=outframe)
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)
end = time.time()
print("Frames per second using OpenCV:  " + str(iterations / (end - start)))

运行成功后，<br>在输入 1920 * 1080 的信号时,每秒的帧率大约在3帧左右。

## 步骤4：采用cv2pynq中的Sobel算子对输入信号进行处理

In [ ]:
import time

iterations = 10

start = time.time()
for i in range(iterations):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    cv2.Sobel(inframe,-1,1,0,ksize=5,dst=outframe)
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)
end = time.time()
print("Frames per second using cv2PYNQ:  " + str(iterations / (end - start)))

在采用相同的输入信号(1920* 1080),相同的Sobel算子下，<br>通过cv2pynq输出的视频流信号帧率在60帧左右。

## 步骤5：释放HDMI驱动

In [ ]:
hdmi_out.close()
hdmi_in.close()

## 步骤6：关闭cv2pynq

关闭cv2pynq是一个很重要的步骤，因为在BaseOverlay中的video子系统模块中，图片是以连续的内存数组（contiguous memory arrays）作为存储形式，因此在调用cv2pynq时，可以直接将数据以流的形式传输到PL端。所以避免cv2pynq一直占用连续的内存，必须关于cv2pynq以释放内存，而对连续的内存分配是基于PYNQ的Xlnk库，关于Xlnk的更多详情，可参考：<br>https://pynq.readthedocs.io/en/latest/pynq_libraries/xlnk.html

In [ ]:
cv2.close()

## 附录：PL端是如何加速OPENCV函数处理的

在此项目中式采用一种基于Vivado HLS加速OpenCV程序的方法:<br>其核心是利用Xilinx高层次综合工具Vivado HLS，将C++编写的OpenCV程序按照Vivado HLS处理规范进行修改，进而将代码转换为硬件描述语言，可快速生成IP核。结合Xilinx PYNQ SoC架构,在顶层可直接对我们的Block Design进行Python式的封装，实现OpenCV程序算法向SoC系统的移植和加速。<br>

![Image of HLS](../image/2.png)

#### Sobel算子概述

Sobel算子是像素图像边缘检测中最重要的算子之一，在机器学习、数字媒体、计算机视觉等信息科技领域起着举足轻重的作用。在技术上，它是一个离散的一阶差分算子，用来计算图像亮度函数的一阶梯度之近似值。在图像的任何一点使用此算子，将会产生该点对应的梯度矢量或是其法矢量。

#### Sobel算子核心公式

该算子包含两组3x3的矩阵（当ksize=3时），分别为横向及纵向，将之与图像作平面卷积，即可分别得出横向及纵向的亮度差分近似值。如果以A代表原始图像，Gx及Gy分别代表经横向及纵向边缘检测的图像，其公式如下:

![Image of HLS1](../image/sobel1.png)

图像的每一个像素的横向及纵向梯度近似值可用以下的公式结合，来计算梯度的大小。 

![Image of HLS2](../image/sobel2.png)

然后可用以下公式计算梯度方向：

![Image of HLS3](../image/sobel3.png)<br>

更多关于Sobel算子的详细信息，可参考：<br>https://docs.opencv.org/3.0-beta/doc/tutorials/imgproc/imgtrans/sobel_derivatives/sobel_derivatives.html

#### 在Vivado HLS中映射Sobel算子的结构

在OpenCV中，通过传入dx与dy来求X方向的梯度以及Y方向的梯度从而输出不同方向上的处理结果
而在Vivado HLS中，此工程建立了一个可通用性的卷积核矩阵IP核(fliter2D)，通过接受传输参数来控制卷积核的参数

>def Sobel(self,src, ddepth, dx, dy, dst, ksize):<br>
&ensp;&ensp;&ensp;&ensp;if(ksize == 3):<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.f2D.rows = src.shape[0]<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.f2D.columns = src.shape[1]<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.f2D.channels = 1<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;if (dx == 1) and (dy == 0) :<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;if self.filter2DType != 0 :<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.filter2DType = 0<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.f2D.r1 = 0x000100ff #[-1  0  1]<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.f2D.r2 = 0x000200fe #[-2  0  2]<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;self.f2D.r3 = 0x000100ff #[-1  0  1]<br>

上述代码为顶层Python封装时对Sobel函数的部分描述，从中可以获得信息：<br>顶层通过传递dx与dy的值，设置好卷积核的参数，传入IP核（fliter2D）中，下述代码为在Vivado HLS中对IP核的部分描述：

>&ensp;&ensp;&ensp;&ensp;GRAY_IMAGE g_img_0(rows,cols);<br>
&ensp;&ensp;&ensp;&ensp;GRAY_IMAGE g_img_1(rows,cols);<br>
&ensp;&ensp;&ensp;&ensp;ap_uint<32> dat = in_stream->data;<br>
&ensp;&ensp;&ensp;&ensp;g_img_0.write(GRAY_PIXEL(dat.range(7,0)))<br><br>
&ensp;&ensp;&ensp;&ensp;kernel.val[0][0] = r1.range(7,0);<br>
&ensp;&ensp;&ensp;&ensp;kernel.val[1][0] = r2.range(7,0);<br>
&ensp;&ensp;&ensp;&ensp;kernel.val[2][0] = r3.range(7,0);<br><br>
&ensp;&ensp;&ensp;&ensp;hls::Filter2D(g_img_0,g_img_1,kernel, c_point);<br><br>
&ensp;&ensp;&ensp;&ensp;dat.range(7,0) = g_img_1.read().val[0];<br>
&ensp;&ensp;&ensp;&ensp;out_stream->data = dat;

从Vivado HLS中对IP核的部分描述中，可以得到以下信息：<br>
* 将输入的信息用g_img_0来存储
* 根据PS端传入的r1,r2,r3参数设置卷积核
* 将g_img_0与设置好的卷积核(kernel)进行卷积，卷积结果输出给g_img_1
* 将输出结果赋予out_stream

上述过程简要的描述了Sobel算子在(ksize = 3)的情况下，如何在HLS中编写相应的算法从而生成IP核，并且在上层用Python对IP核进行封装的过程。<br>如要了解更多的关于OpenCV在HLS上的应用，可以参考XAP1167。<br>如需对本UserGuide中的源码有更多的了解，可以参考：<br>https://github.com/wbrueckner/cv2pynq/blob/master/cv2pynq/cv2pynq.py<br>
https://github.com/wbrueckner/cv2PYNQ-The-project-behind-the-library/blob/master/ip/HLS/filter2D/filter2D_hls.cpp